In [1]:
import pandas as pd
import numpy as np

In [2]:
clean_df = pd.read_csv('data/clean_data.csv')
clean_df

,Names,Genders,City,17-25,26-34,35-43,44-52,52+,Math Tutor,Science Tutor,...,CSN1,CSN2,CSN3,CSN4,CSN5,OPN1,OPN2,OPN3,OPN4,OPN5
0,Novia Rizki Wulandari,1,Bandung,1,0,0,0,0,1,0,...,5,4,4,5,5,4,4,2,4,4
1,Nabhan Nabilah,1,Bandung,1,0,0,0,0,0,0,...,5,2,5,5,5,2,2,5,5,2
2,Herlina Kusyanuri Putri,1,Bandung,1,0,0,0,0,0,0,...,5,5,5,5,5,4,3,3,4,3
3,Rifdah Alyaa,1,Makassar,1,0,0,0,0,1,0,...,2,3,2,3,4,3,2,3,4,3
4,Salsabilla,1,Bandung,1,0,0,0,0,0,1,...,4,3,3,4,4,2,2,2,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Usamah,0,Bandung,1,0,0,0,0,0,0,...,5,5,4,5,5,4,4,5,5,5
97,Helena,1,Bandung,0,0,0,0,0,1,0,...,4,3,4,4,4,3,4,2,5,4
98,Deva,1,Bandung,0,1,0,0,0,0,0,...,4,4,4,3,4,3,4,2,4,3
99,Dzaky,0,Bandung,1,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,4,5
